In [1]:
import argparse
import pickle
from pathlib import Path

import fraud_eagle as feagle
import fraudar
import networkx as nx
import numpy as np
import pandas as pd
import rsd

from rev2 import rev2compute
from utils import load_data, split_data_by_time

import seaborn as sns
import matplotlib.pyplot as plt

import sklearn.metrics

import warnings
from sklearn.exceptions import UndefinedMetricWarning
warnings.filterwarnings(action='ignore', category=UndefinedMetricWarning)

In [2]:
def get_results(data_name="alpha", alg="fraudar", attack_name="ilp_attack", budget=100, frac=0.2):
    data_nw_df, data_gt_df = load_data(data_name=data_name)
    p = Path(f"../res/{attack_name}/{alg}-{data_name}/{budget}-{frac}.pkl")
    if not p.exists():
        print(p)
        return None
    with open(p, "rb") as fp:
        scores = pickle.load(fp)
        scores = dict(zip(range(len(scores)), scores))
        # 1 means fraudster, 0 means benign
        scores["truth"] = dict(zip(data_gt_df["id"], (1-data_gt_df["label"])/2))
    # df = pd.DataFrame({sp: {u: scores[sp][u] for u in scores[sp] if u in data_gt_df["id"].tolist()} for sp in scores})
    df = pd.DataFrame({sp: {u: scores[sp][u] for u in scores[sp]} for sp in scores})
    df.loc[[u[:5]=="usock" for u in df.index.tolist()], "truth"] = 1
    df.loc[[u[:6]=="udummy" for u in df.index.tolist()], "truth"] = 1
    return df

In [3]:
def get_metrics(df_res, splits=4, K=None, th=0.5):
    if df_res is None:
        return None
    ret = {}
    for sp in range(splits):
        # df_res["truth"] = df_res["truth"].fillna(1)
        df_res.loc[[u[:5]=="usock" for u in df_res.index.tolist()], "truth"] = 1
        df_res.loc[[u[:6]=="udummy" for u in df_res.index.tolist()], "truth"] = 1
        df = df_res[[sp, "truth"]].dropna()
        df[sp]
        t = df["truth"].sum() if K is None else K
        t_frac = t / df.shape[0]
        df["pred"] = df[sp] >= th

        ret[sp] = {
            "precision": sklearn.metrics.precision_score(y_true=df["truth"], y_pred=df["pred"]),
            "recall": sklearn.metrics.recall_score(y_true=df["truth"], y_pred=df["pred"]),
            "f1": sklearn.metrics.f1_score(y_true=df["truth"], y_pred=df["pred"])
        }
    return ret

In [12]:
alg_list = ["fraudar", "rsd", "rev2", "sg", "fbox"][:4]
data_list = ["alpha", "otc", "amazon", "epinions"][:2]
budget_list = [100.0, 200.0, 300.0, 400.0]
frac_list = [0.5, 0.7, 0.9]

In [13]:
attack_res = {
    (d, a, b, f): get_results(data_name=d, alg=a, attack_name="rtv_attack", budget=b, frac=f)
    for d in data_list for a in alg_list
    for b in budget_list for f in frac_list
}

In [14]:
# attack_res[("epinions", "fraudar", 400.0, 0.2)].to_csv("ar.csv")

In [15]:
th_dict = {
    "fraudar": 0.5,
    "rsd": 1.2,
    "rev2": 0.1,
    "fbox": 0.5,
    "sg": 0.5,
}
dict_met = {
    (d, a, b, f): pd.DataFrame.from_dict(get_metrics(df_res=attack_res[(d, a, b, f)], splits=4, K=None, th=th_dict[a]))
    for d, a, b, f in attack_res
}
df_met = pd.concat(dict_met, keys=dict_met.keys())
display(df_met)

0         1         2         3
alpha fraudar 100.0 0.5 precision  0.357513  0.382199  0.380682  0.356250
                        recall     0.650943  0.657658  0.638095  0.575758
                        f1         0.461538  0.483444  0.476868  0.440154
                    0.7 precision  0.357513  0.378947  0.380682  0.356250
                        recall     0.584746  0.585366  0.572650  0.513514
...                                     ...       ...       ...       ...
otc   sg      400.0 0.7 recall     0.053004  0.050000  0.051282  0.000000
                        f1         0.085227  0.082596  0.086420  0.000000
                    0.9 precision  0.205882  0.237288  0.260000  0.000000
                        recall     0.042296  0.042683  0.040498  0.000000
                        f1         0.070175  0.072351  0.070081  0.000000

[288 rows x 4 columns]

In [20]:
def get_alg_data_met(df_met, alg="rev2", data="alpha", met="f1"):
    df_met.index = df_met.index.set_names(["data", "algo", "budget", "frac", "metrics"])
    rdf = df_met.mean(axis=1).to_frame().reset_index().pivot(index=["data", "algo", "frac", "metrics"], columns=["budget"]).loc[(data, alg, slice(None), met)].droplevel(0, axis=1).droplevel([0, 1, 3], axis=0)
    rdf.index = [f"baseline-{i}" if i > 0 else "no-attack" for i in rdf.index]
    return rdf

In [43]:
get_alg_data_met(df_met, alg="sg", data="otc", met="precision")

budget,100.0,200.0,300.0,400.0
baseline-0.5,0.178670,0.181521,0.182297,0.181521
baseline-0.7,0.175793,0.181521,0.181521,0.182297
baseline-0.9,0.175793,0.175793,0.178670,0.175793
